In [1]:
DATA_PREPARATION_VOTE_METHOD = "max_vote_window" # "max_vote_window" or "sum_and_normalize". Decides how to aggregate the predictions of the overlapping windows

In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm 
import wandb
from torch.optim.lr_scheduler import CosineAnnealingLR
import sys

if bool(os.environ.get("KAGGLE_URL_BASE", "")):
  import sys
  # running on kaggle
  sys.path.insert(0, "/kaggle/input/hsm-source-files")
else:
  # running locally
  sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..")))

from src.datasets.cbramod_dataset import CBraModDataset
from src.utils.k_folds_creator import KFoldCreator
from src.utils.utils import get_models_save_path, set_seeds, get_raw_data_dir, get_processed_data_dir
from src.models.cbramod_model import CBraModModel
from src.utils.constants import Constants 
from src.datasets.eeg_processor import EEGDataProcessor
from huggingface_hub import hf_hub_download

set_seeds(Constants.SEED)

/home/david/miniconda3/envs/aicomp/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/david/miniconda3/envs/aicomp/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened bec

Skipping module tcn due to missing dependency: No module named 'pytorch_tcn'


In [3]:
wandb.login()

wandb: Currently logged in as: davidhodel (hms-hslu-aicomp-hs25) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
DATA_PATH = get_raw_data_dir()

processor = EEGDataProcessor(raw_data_path=DATA_PATH, processed_data_path=get_processed_data_dir())
train_df = processor.process_data(vote_method=DATA_PREPARATION_VOTE_METHOD, skip_parquet=True)

kl_score = nn.KLDivLoss(reduction="batchmean")

Processor initialized.
Raw data path: '/home/david/git/aicomp/data'
Processed data path: '/home/david/git/aicomp/data/processed'
Starting EEG Data Processing Pipeline
Skipping Parquet file creation as requested.
Using 'max_vote_window' vote aggregation strategy.

Processed train data saved to '/home/david/git/aicomp/data/processed/train_processed.csv'.
Shape of the final dataframe: (17089, 12)

Pipeline finished successfully!


In [5]:
pretrained_weights_path = hf_hub_download("weighting666/CBraMod", "pretrained_weights.pth", repo_type="model")
print(f"Pretrained weights downloaded to: {pretrained_weights_path}")

Pretrained weights downloaded to: /home/david/.cache/huggingface/hub/models--weighting666--CBraMod/snapshots/500543c7e30bda1b22bfd51a49301b238dee21fd/pretrained_weights.pth


In [6]:
fold_creator = KFoldCreator(n_splits=5, seed=Constants.SEED)
train_df = fold_creator.create_folds(train_df, stratify_col='expert_consensus', group_col='patient_id')

In [7]:
class Config:
  window_size_seconds = 10
  eeg_normalization = "naive"
  batch_size = 64
  num_dataset_workers = 8
  dropout_prob = 0.1
  learning_rate = 1*10**(-4.5)
  weight_decay = 5e-2
  num_epochs = 50
  clip_grad_norm = 1.0

class WandbConfig:
  entity = "hms-hslu-aicomp-hs25"
  project_name = "hms-aicomp-cbramod"
  run_name = f"cbramod-finetune_window_{Config.window_size_seconds}s_bs{Config.batch_size}_lr{Config.learning_rate}_wd{Config.weight_decay}_dropout{Config.dropout_prob}_norm-{Config.eeg_normalization}"

In [8]:
def get_dataloaders(df, fold_id):
    fold_train_df = df[df['fold'] != fold_id].reset_index(drop=True)
    fold_valid_df = df[df['fold'] == fold_id].reset_index(drop=True)

    train_dataset = CBraModDataset(fold_train_df, DATA_PATH, Config.window_size_seconds, eeg_frequency=200, mode='train', normalization=Config.eeg_normalization)
    valid_dataset = CBraModDataset(fold_valid_df, DATA_PATH, Config.window_size_seconds, eeg_frequency=200, mode='train', normalization=Config.eeg_normalization)

    train_loader = DataLoader(
        train_dataset,
        batch_size=Config.batch_size,
        shuffle=True,
        num_workers=Config.num_dataset_workers,
        pin_memory=True,
        drop_last=True,
    )
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=Config.batch_size,
        shuffle=False,
        num_workers=Config.num_dataset_workers,
        pin_memory=True,
        drop_last=False,
    )

    return train_loader, valid_loader

In [9]:
# train_loader, valid_loader = get_dataloaders(train_df, fold_id=0)
# first_batch = next(iter(train_loader))
# eegs, labels = first_batch
# print(f"EEG batch shape: {eegs.shape}, labels batch shape: {labels.shape}") # batch_size, num_channels, seq_len, patch_size

In [10]:
def create_model(device):
  model = CBraModModel(
    pretrained_weights_path=pretrained_weights_path,
    classifier_type="all_patch_reps",
    num_of_classes=len(Constants.TARGETS),
    dropout_prob=Config.dropout_prob,
    num_eeg_channels=len(Constants.EEG_FEATURES),
    seq_len_seconds=Config.window_size_seconds,
    device=device,
  ).to(device)

  backbone_params = []
  other_params = []
  for name, param in model.named_parameters():
      if "backbone" in name:
          backbone_params.append(param)
          param.requires_grad = True
      else:
          other_params.append(param)

  return model, backbone_params, other_params

def create_optimizer(backbone_params, other_params, lr, weight_decay):
  optimizer = torch.optim.AdamW([
      {'params': backbone_params, 'lr': lr},
      {'params': other_params, 'lr': lr * 5}
  ], weight_decay=weight_decay)
  return optimizer

def create_lr_scheduler(optimizer, data_length, num_epochs):
  scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs*data_length, eta_min=1e-6)
  return scheduler

In [ ]:
loss_fn = nn.KLDivLoss(reduction='batchmean')

def run_training():
  device = torch.device("cuda")

  all_oof_preds = []
  all_oof_labels = []
  for fold_id in range(fold_creator.n_splits):
    print(f"\n========== FOLD {fold_id} ==========")
    torch.cuda.empty_cache()

    run = wandb.init(
        entity=WandbConfig.entity,
        project=WandbConfig.project_name,
        name=f"{WandbConfig.run_name}_fold{fold_id}", 
        tags=[f'fold{fold_id}'],
        config= {
            "window_size_seconds": Config.window_size_seconds,
            "eeg_normalization": Config.eeg_normalization,
            "batch_size": Config.batch_size,
            "dropout_prob": Config.dropout_prob,
            "learning_rate": Config.learning_rate,
            "weight_decay": Config.weight_decay,
            "num_epochs": Config.num_epochs,
            "clip_grad_norm": Config.clip_grad_norm,
            "data_preparation_vote_method": DATA_PREPARATION_VOTE_METHOD,
        }
    )


    model, backbone_params, other_params = create_model(device)

    optimizer = create_optimizer(backbone_params, other_params, Config.learning_rate, Config.weight_decay)
    train_loader, valid_loader = get_dataloaders(train_df, fold_id)
    scheduler = create_lr_scheduler(optimizer, len(train_loader), Config.num_epochs)

    best_val_loss = float('inf')
    best_model_path = None

    for epoch in range(Config.num_epochs):
      model.train()
      train_loss = 0

      for eeg_windows, labels in tqdm(train_loader, desc=f"Fold {fold_id} Epoch {epoch} Training"):
          eeg_windows, labels = eeg_windows.to(device), labels.to(device)

          optimizer.zero_grad()
          outputs = model(eeg_windows)
          log_probs = F.log_softmax(outputs, dim=1)
          loss = loss_fn(log_probs, labels)
          loss.backward()
          if Config.clip_grad_norm > 0:
              torch.nn.utils.clip_grad_norm_(model.parameters(), Config.clip_grad_norm)
          optimizer.step()
          scheduler.step()

          train_loss += loss.item() * eeg_windows.size(0)

          run.log({
             "train/loss": loss.item(),
             "train/lr": scheduler.get_last_lr()[0],
          })

      train_loss /= len(train_loader.dataset)

      model.eval()
      valid_loss = 0
      with torch.no_grad():
        for eeg_windows, labels in tqdm(valid_loader, desc=f"Fold {fold_id} Epoch {epoch} Validation"):
            eeg_windows, labels = eeg_windows.to(device), labels.to(device)

            outputs = model(eeg_windows)
            log_probs = F.log_softmax(outputs, dim=1)
            loss = loss_fn(log_probs, labels)

            valid_loss += loss.item() * eeg_windows.size(0)

      valid_loss /= len(valid_loader.dataset)
      print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Valid Loss = {valid_loss:.4f}")

      run.log({
          "epoch": epoch + 1,
          "train/epoch_loss": train_loss,
          "val/loss": valid_loss,
          "val/kl_div": valid_loss,
      })

      if valid_loss < best_val_loss:
          best_val_loss = valid_loss
          best_model_path = get_models_save_path() / "cbramod" / DATA_PREPARATION_VOTE_METHOD / f"best_model_fold_{fold_id}.pth"
          best_model_path.parent.mkdir(parents=True, exist_ok=True)
          torch.save(model.state_dict(), best_model_path)

      if best_model_path is not None:
          model.load_state_dict(torch.load(best_model_path))
          model.eval()

          fold_oof_preds = []
          fold_oof_labels = []

          with torch.no_grad():
            for eeg_windows, labels in tqdm(valid_loader, desc=f"Fold {fold_id} OOF Predictions"):
                eeg_windows = eeg_windows.to(device)

                outputs = model(eeg_windows)
                probs = F.softmax(outputs, dim=1).cpu().numpy()

                fold_oof_preds.append(probs)
                fold_oof_labels.append(labels.numpy())

          all_oof_preds.append(np.concatenate(fold_oof_preds, axis=0))
          all_oof_labels.append(np.concatenate(fold_oof_labels, axis=0))
      else:
          raise RuntimeError("Best model path is None, cannot generate OOF predictions.")
      
    if best_model_path is not None:
        artifact = wandb.Artifact(f"{run.name}-{run.id}", type='model')
        artifact.add_file(str(best_model_path))
        run.log_artifact(artifact)
    else:
       raise RuntimeError("Best model path is None, cannot log model artifact.")

    run.summary['best_val_kl_div'] = best_val_loss
    run.finish()
    break # used to debugging for only one fold

  if all_oof_preds and all_oof_labels:
      print("\nCalculating final OOF score...")
      final_oof_preds = np.concatenate(all_oof_preds)
      final_oof_labels = np.concatenate(all_oof_labels)

      oof_preds_tensor = torch.tensor(final_oof_preds, dtype=torch.float32)
      oof_labels_tensor = torch.tensor(final_oof_labels, dtype=torch.float32)

      log_oof_preds_tensor = torch.log(oof_preds_tensor + 1e-8)

      overall_oof_score = loss_fn(log_oof_preds_tensor, oof_labels_tensor).item()

      print(f"\nOverall OOF KL Score: {overall_oof_score:.4f}")
  else:
      print("\nCould not calculate OOF score because no predictions were generated.")
      
  return overall_oof_score

run_training()


========== FOLD 0 ==========


Fold 0 Epoch 0 Validation: 100%|██████████| 64/64 [00:05<00:00, 11.44it/s]


Epoch 0: Train Loss = 1.0266, Valid Loss = 1.3498


Fold 0 Epoch 1 Validation: 100%|██████████| 64/64 [00:04<00:00, 13.01it/s]


Epoch 1: Train Loss = 0.8273, Valid Loss = 1.3408


...

Fold 0 Epoch 40 Validation: 100%|██████████| 64/64 [00:06<00:00, 10.32it/s]


Epoch 40: Train Loss = 0.4325, Valid Loss = 1.1933


Fold 0 Epoch 41 Validation: 100%|██████████| 64/64 [00:05<00:00, 10.84it/s]


Epoch 41: Train Loss = 0.4341, Valid Loss = 1.1779


Fold 0 Epoch 42 Validation:  27%|██▋       | 17/64 [00:02<00:04,  9.93it/s]